# Olympic Medal Case Study Submission:

## Part 1: Web Scrape

In [1]:
# Import Libraries:
import numpy as np
import pandas as pd
from sqlalchemy import create_engine

In [2]:
# Scrape table from URL: 
# Pandas surpasses BeautifulSoup here and delivers a list of DataFrames from the read_html() method.
url = 'https://en.wikipedia.org/wiki/2018_Winter_Olympics_medal_table'
html = pd.read_html(url)

# Table for this case study is located at index 1 of the list:
df = html[1]

In [3]:
# Verify table output:
df

,Rank,NOC,Gold,Silver,Bronze,Total
0,1,Norway (NOR),14,14,11,39
1,2,Germany (GER),14,10,7,31
2,3,Canada (CAN),11,8,10,29
3,4,United States (USA),9,8,6,23
4,5,Netherlands (NED),8,6,6,20
5,6,Sweden (SWE),7,6,1,14
6,7,South Korea (KOR)*,5,8,4,17
7,8,Switzerland (SUI),5,6,4,15
8,9,France (FRA),5,4,6,15
9,10,Austria (AUT),5,3,6,14


In [4]:
# Remove bottom row of column totals to not interfere with SQL Queries:
df = df.iloc[:-1].copy()

In [5]:
# Examine datatypes & Null values: No correction is needed
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30 entries, 0 to 29
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Rank    30 non-null     object
 1   NOC     30 non-null     object
 2   Gold    30 non-null     int64 
 3   Silver  30 non-null     int64 
 4   Bronze  30 non-null     int64 
 5   Total   30 non-null     int64 
dtypes: int64(4), object(2)
memory usage: 1.5+ KB


## Part 2: SQL Queries

In [6]:
# Create local engine and SQL database for case study purposes:
# Table is named 'medal_counts' for querying.
engine = create_engine('sqlite://')
sql_data = df.to_sql('medal_counts',con=engine)

#### SQL Query A:

In [7]:
# Queries are made with pandas read_sql method for output display:

sql1 = '''

SELECT NOC
FROM medal_counts
WHERE Bronze > Gold
ORDER BY Bronze DESC

'''
# Return query output: 
pd.read_sql(sql1,con=engine)

,NOC
0,Olympic Athletes from Russia (OAR)
1,France (FRA)
2,Austria (AUT)
3,Italy (ITA)
4,Finland (FIN)
5,Great Britain (GBR)
6,Czech Republic (CZE)
7,China (CHN)
8,New Zealand (NZL)
9,Spain (ESP)


#### SQL Query B:

In [8]:
sql2 = '''

SELECT DISTINCT table1.NOC
FROM medal_counts AS table1, medal_counts AS table2
WHERE table1.Total = table2.Total
AND table1.NOC != table2.NOC
ORDER BY table1.Total DESC

'''
# Return query output: 
pd.read_sql(sql2,con=engine)

,NOC
0,South Korea (KOR)*
1,Olympic Athletes from Russia (OAR)
2,Switzerland (SUI)
3,France (FRA)
4,Sweden (SWE)
5,Austria (AUT)
6,Belarus (BLR)
7,Slovakia (SVK)
8,Australia (AUS)
9,Poland (POL)


#### SQL Query C:

In [9]:
sql3 = '''

SELECT Rank, NOC AS Gold, Silver, Bronze
FROM (SELECT Gold, NOC,
    RANK() OVER (ORDER BY Gold DESC, NOC) AS Rank
    FROM medal_counts) AS gold_table

    JOIN (SELECT Rank1, NOC AS Silver
        FROM (SELECT Silver, NOC,
        RANK() OVER (ORDER BY Silver DESC, NOC) AS Rank1
        FROM medal_counts)) AS silver_table
    ON gold_table.Rank = silver_table.Rank1
    
    JOIN (SELECT Rank2, NOC AS Bronze
        FROM (SELECT Bronze, NOC,
        RANK() OVER (ORDER BY Bronze DESC, NOC) AS Rank2
        FROM medal_counts)) AS bronze_table
    ON gold_table.Rank = bronze_table.Rank2

'''
# Return query output:
pd.read_sql(sql3,con=engine)

,Rank,Gold,Silver,Bronze
0,1,Germany (GER),Norway (NOR),Norway (NOR)
1,2,Norway (NOR),Germany (GER),Canada (CAN)
2,3,Canada (CAN),Canada (CAN),Olympic Athletes from Russia (OAR)
3,4,United States (USA),South Korea (KOR)*,Germany (GER)
4,5,Netherlands (NED),United States (USA),Austria (AUT)
5,6,Sweden (SWE),China (CHN),France (FRA)
6,7,Austria (AUT),Netherlands (NED),Netherlands (NED)
7,8,France (FRA),Olympic Athletes from Russia (OAR),United States (USA)
8,9,South Korea (KOR)*,Sweden (SWE),Italy (ITA)
9,10,Switzerland (SUI),Switzerland (SUI),Finland (FIN)
